In [118]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
from datetime import datetime as dt

from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools
import warnings
warnings.filterwarnings("ignore")

In [119]:
fund_df = pd.read_csv("sp500_fundamental_199601_202502.csv")

In [120]:
df_daily_price = pd.read_csv("sp500_price_199601_202502.csv")

In [121]:
df_daily_price['date'] = df_daily_price['date'].astype(str).str[:10]
df_daily_price['date'] = pd.to_datetime(df_daily_price['date'], format='%Y-%m-%d', errors='coerce')

In [122]:
len(df_daily_price.tic.unique())

861

In [123]:
len(fund_df.tic.unique())

924

In [124]:
# fund_df['datadate'] = pd.to_datetime(fund_df['datadate'])
df_daily_price['date'] = pd.to_datetime(df_daily_price['date'])
fund_df['datadate'] = (pd.to_datetime(fund_df['datadate']).dt.strftime('%Y%m%d')).astype(int)

In [125]:
trading_dates = list(df_daily_price['date'].unique())

In [126]:
def check_trading_date(date):
    # date = date.strftime('%Y%m%d').astype(int)
    while date not in trading_dates:
        date += + pd.Timedelta(days=1)
    # return pd.to_datetime(date,format='%Y%m%d')
    return date

In [127]:
# fund_df['year'] = (fund_df['datadate'].str[:4] + fund_df['datadate'].str[5:7] + fund_df.str[-2:]).astype(int)
# use trade date instead of report quarterly date
times = list(fund_df['datadate']) # quarterly report date
for i in range(len(times)):
    quarter = (times[i] - int(times[i]/10000)*10000)
    if 1201 < quarter:
        times[i] = int(times[i]/10000 + 1)*10000 + 301
    if quarter <= 301:
        times[i] = int(times[i]/10000)*10000 + 301
    if 301 < quarter <= 601:
        times[i] = int(times[i]/10000)*10000 + 601
    if 601 < quarter <= 901:
        times[i] = int(times[i]/10000)*10000 + 901
    if 901 < quarter <= 1201:
        times[i] = int(times[i]/10000)*10000 + 1201
    # times[i] = check_trading_date(times[i])
    time_tmp = times[i]
    year = int(time_tmp/10000)
    month = int(quarter/100)
    day = int(quarter - month*100)
#     if(time_tmp < 20171114):
#         while(dt.date(year, month, day).weekday() > 4):
#             time_tmp = time_tmp + 1
#             year = int(time_tmp/10000)
#             month = int((time_tmp - year*10000)/100)
#             day = int(time_tmp - year*10000 - month*100)
#         times[i] = time_tmp
times = pd.to_datetime(times,format='%Y%m%d')
fund_df['datadate']=times

In [128]:
fund_times = list(sorted(times.unique()))
date_map_df = pd.DataFrame()
date_map_df['report_date'] = fund_times[:-1]
date_map_df['trading_date'] = date_map_df['report_date'].apply(check_trading_date)
fund_df = pd.merge(fund_df, date_map_df, left_on='datadate',right_on='report_date')
fund_df.drop(columns=['datadate','report_date'], inplace=True)
fund_df.rename(columns={'trading_date':'datadate'}, inplace=True)

In [129]:
fund_df = fund_df.merge(df_daily_price, left_on=['datadate', 'tic'], right_on=['date', 'tic'], how='left')
fund_df.drop(columns=['datadate'], inplace=True)

In [130]:
l_df = list(fund_df.groupby('tic'))
for tic,df in l_df:
    df.reset_index(inplace=True, drop=True)
    df.sort_values('date')
    # our goal is to predict next quarter's return
    df['y_return'] = np.log(df['adj_close_q'].shift(-1) / df['adj_close_q'])

In [131]:
fund_df = pd.concat([x[1] for x in l_df])

In [132]:
fund_df['pe'] = fund_df.prccq / fund_df.epspxq
fund_df['ps'] = fund_df.prccq / (fund_df.revtq/fund_df.cshoq)
fund_df['pb'] = fund_df.prccq / ((fund_df.atq-fund_df.ltq)/fund_df.cshoq)

In [133]:
items = [
    'date', # Date
    'gvkey',#gvkey unique identifier
    'tic', # Ticker
    'gsector',#gics sector 11
    'oiadpq', # Quarterly operating income
    'revtq', # Quartely revenue
    'niq', # Quartely net income
    'atq', # Total asset
    'teqq', # Shareholder's equity
    'epspiy', # EPS(Basic) incl. Extraordinary items
    'ceqq', # Common Equity
    'cshoq', # Common Shares Outstanding
    'dvpspq', # Dividends per share
    'actq', # Current assets
    'lctq', # Current liabilities
    'cheq', # Cash & Equivalent
    'rectq', # Recievalbles
    'cogsq', # Cost of  Goods Sold
    'invtq', # Inventories
    'apq',# Account payable
    'dlttq', # Long term debt
    'dlcq', # Debt in current liabilites
    'ltq', # Liabilities   
    'pe', #Price–earnings ratio
    'ps', #Price–sales ratio
    'pb', #Price-to-Book (P/B) Ratio
    'adj_close_q',#adjusted close price
    'y_return' #next quarter return
]

# Omit items that will not be used
fund_data = fund_df[items]


In [134]:
# Rename column names for the sake of readability
fund_data = fund_data.rename(columns={
    'oiadpq':'op_inc_q', # Quarterly operating income
    'revtq':'rev_q', # Quartely revenue
    'niq':'net_inc_q', # Quartely net income
    'atq':'tot_assets', # Assets
    'teqq':'sh_equity', # Shareholder's equity
    'epspiy':'eps_incl_ex', # EPS(Basic) incl. Extraordinary items
    'ceqq':'com_eq', # Common Equity
    'cshoq':'sh_outstanding', # Common Shares Outstanding
    'dvpspq':'div_per_sh', # Dividends per share
    'actq':'cur_assets', # Current assets
    'lctq':'cur_liabilities', # Current liabilities
    'cheq':'cash_eq', # Cash & Equivalent
    'rectq':'receivables', # Receivalbles
    'cogsq':'cogs_q', # Cost of  Goods Sold
    'invtq':'inventories', # Inventories
    'apq': 'payables',# Account payable
    'dlttq':'long_debt', # Long term debt
    'dlcq':'short_debt', # Debt in current liabilites
    'ltq':'tot_liabilities', # Liabilities   
})

In [135]:
# set data type to series
date = fund_data['date'].to_frame('date').reset_index(drop=True)
tic = fund_data['tic'].to_frame('tic').reset_index(drop=True)
gvkey = fund_data['gvkey'].to_frame('gvkey').reset_index(drop=True)
adj_close_q = fund_data['adj_close_q'].to_frame('adj_close_q').reset_index(drop=True)
y_return = fund_data['y_return'].to_frame('y_return').reset_index(drop=True)
gsector = fund_data['gsector'].to_frame('gsector').reset_index(drop=True)
pe = fund_data['pe'].to_frame('pe').reset_index(drop=True)
ps = fund_data['ps'].to_frame('ps').reset_index(drop=True)
pb = fund_data['pb'].to_frame('pb').reset_index(drop=True)

In [136]:
# Calculate financial ratios

# Profitability ratios
# Operating Margin
OPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='OPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        OPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        OPM.iloc[i] = np.nan
    else:
        OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
OPM=pd.Series(OPM).to_frame().reset_index(drop=True)

# Net Profit Margin        
NPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='NPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        NPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        NPM.iloc[i] = np.nan
    else:
        NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
NPM=pd.Series(NPM).to_frame().reset_index(drop=True)

# Return On Assets
ROA = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROA')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROA[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROA.iloc[i] = np.nan
    else:
        ROA.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['tot_assets'].iloc[i]
ROA=pd.Series(ROA).to_frame().reset_index(drop=True)

# Return on Equity
ROE = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROE')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROE[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROE.iloc[i] = np.nan
    else:
        ROE.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['sh_equity'].iloc[i]        
ROE=pd.Series(ROE).to_frame().reset_index(drop=True)

# For calculating valuation ratios in the next subpart, calculate per share items in advance
# Earnings Per Share       
EPS = fund_data['eps_incl_ex'].to_frame('EPS').reset_index(drop=True)

# Book Per Share
BPS = (fund_data['com_eq']/fund_data['sh_outstanding']).to_frame('BPS').reset_index(drop=True) # Need to check units

#Dividend Per Share
DPS = fund_data['div_per_sh'].to_frame('DPS').reset_index(drop=True)

# Liquidity ratios
# Current ratio
cur_ratio = (fund_data['cur_assets']/fund_data['cur_liabilities']).to_frame('cur_ratio').reset_index(drop=True)

# Quick ratio
quick_ratio = ((fund_data['cash_eq'] + fund_data['receivables'] )/fund_data['cur_liabilities']).to_frame('quick_ratio').reset_index(drop=True)

# Cash ratio
cash_ratio = (fund_data['cash_eq']/fund_data['cur_liabilities']).to_frame('cash_ratio').reset_index(drop=True)


# Efficiency ratios
# Inventory turnover ratio
inv_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='inv_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        inv_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        inv_turnover.iloc[i] = np.nan
    else:
        inv_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['inventories'].iloc[i]
inv_turnover=pd.Series(inv_turnover).to_frame().reset_index(drop=True)

# Receivables turnover ratio       
acc_rec_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_rec_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_rec_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_rec_turnover.iloc[i] = np.nan
    else:
        acc_rec_turnover.iloc[i] = np.sum(fund_data['rev_q'].iloc[i-3:i])/fund_data['receivables'].iloc[i]
acc_rec_turnover=pd.Series(acc_rec_turnover).to_frame().reset_index(drop=True)

# Payable turnover ratio
acc_pay_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_pay_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_pay_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_pay_turnover.iloc[i] = np.nan
    else:
        acc_pay_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['payables'].iloc[i]
acc_pay_turnover=pd.Series(acc_pay_turnover).to_frame().reset_index(drop=True)

## Leverage financial ratios
# Debt ratio
debt_ratio = (fund_data['tot_liabilities']/fund_data['tot_assets']).to_frame('debt_ratio').reset_index(drop=True)

# Debt to Equity ratio
debt_to_equity = (fund_data['tot_liabilities']/fund_data['sh_equity']).to_frame('debt_to_equity').reset_index(drop=True)

In [137]:
# Create a dataframe that merges all the ratios
ratios = pd.concat([date,gvkey,tic,gsector,adj_close_q,y_return,OPM,NPM,ROA,ROE,EPS,BPS,DPS,
                    cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,
                   debt_ratio,debt_to_equity,pe,ps,pb], axis=1).reset_index(drop=True)

In [138]:
# Replace NAs infinite values with zero
final_ratios = ratios.copy()
final_ratios = final_ratios.fillna(0)
final_ratios = final_ratios.replace(np.inf,0)

In [139]:
import os

output_dir = 'outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

final_ratios.to_csv('outputs/final_ratios_raw.csv')

In [140]:
final_ratios.head()

,date,gvkey,tic,gsector,adj_close_q,y_return,OPM,NPM,ROA,ROE,...,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,0,10507,0033A,55.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.169339,0.014917,0.000000,0.000000,0.000000,0.695946,0.0,73.883929,6.383067,1.430224
1,0,10507,0033A,55.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.271284,0.010014,0.000000,0.000000,0.000000,0.696722,0.0,47.500000,5.677115,1.461822
2,0,10507,0033A,55.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.266745,0.020813,0.000000,0.000000,0.000000,0.685703,0.0,25.000000,4.625658,1.314168
3,0,10507,0033A,55.0,0.0,0.0,0.333037,0.143130,0.034051,0.0,...,0.207633,0.010034,9.085628,15.509621,8.705043,0.684889,0.0,135.833333,6.244855,1.358811
4,0,10507,0033A,55.0,0.0,0.0,0.312177,0.131014,0.030764,0.0,...,0.313543,0.146420,9.658611,16.798462,11.588044,0.709488,0.0,67.156863,5.149707,1.222281


In [141]:
features_column_financial=[ 'OPM', 'NPM', 'ROA', 'ROE', 'EPS', 'BPS', 'DPS', 'cur_ratio',
       'quick_ratio', 'cash_ratio', 'inv_turnover', 'acc_rec_turnover',
       'acc_pay_turnover', 'debt_ratio', 'debt_to_equity', 'pe', 'ps', 'pb']

In [142]:
def handle_nan(df,features_column_financial):
    ##handle nan, inf
    df=df.drop(list(df[df.adj_close_q==0].index)).reset_index(drop=True)
    df['y_return'] = pd.to_numeric(df['y_return'], errors='coerce')
    for col in features_column_financial:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['y_return'].replace([np.nan,np.inf,-np.inf], np.nan, inplace=True)
    df[features_column_financial].replace([np.nan, np.inf, -np.inf], np.nan, inplace=True)
    dropped_col = []
    for col in features_column_financial:
        if np.any(~np.isfinite(df[col])):
            df.drop(columns=[col], axis=1, inplace=True)
    df.dropna(axis=0, inplace=True)
    df=df.reset_index(drop=True)
    print("dropped_col: ",dropped_col)
    return df

In [143]:
final_ratios=handle_nan(final_ratios,features_column_financial)

dropped_col:  []


In [144]:
final_ratios.date=final_ratios.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [145]:
final_ratios.to_csv('outputs/final_ratios.csv')

In [146]:
for sec, df_ in list(final_ratios.groupby('gsector')):
    df_.to_excel(f"outputs/sector{int(sec)}.xlsx")